<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/Transformers_2023_Flash_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ✨ 1a parte: apresentação do PROBLEMA.
- ##  Na arquitetura Transformers, a self-attention, em sua implementação padrão tem custo $\mathcal{O}(N^2)$, onde $N$ é o comprimento da sequência / entrada do modelo / context-window.

---
---

# Qual a relevância?

- ### De 2017 até 2020/2021 maioria dos Transformers para texto tinham entrada de 512 tokens.
- ### Em 2023 temos GPT4 com 32K tokens, Claude com 100K tokens, LLaMa2 com 4096 tokens  de context window... *O livro The Great Gatsby tem 72K tokens, 210 pages*.
> ### Algumas técnicas:
> 1. ### [ALiBi](https://arxiv.org/abs/2108.12409) $\rightarrow$ Você pode treinar em 2K e fazer finetune em 100K tokens
> 2. ### [Multi Query Attention](https://arxiv.org/pdf/1911.02150.pdf) $\rightarrow$ A  Multi-Head Attention é calculada em batches.
> 3. ### Flash Attention $\rightarrow$ essa apresentação

## Em termos de moneys... 🤑





> ## *O LLaMa 65Bi tem custo estimado em ~ 3Mi em GPU, com contexto de $2048$ tokens. O custo para o mesmo LlaMA com $100$k context-window custaria ~150Mi* 💸 ([fonte](https://blog.gopenai.com/how-to-speed-up-llms-and-use-100k-context-window-all-tricks-in-one-place-ffd40577b4c)).

---
---



# Soluções já abordadas:
> ### [Artigo: Efficient Transformers: A Survey](https://arxiv.org/pdf/2009.06732.pdf)
> <img src="https://drive.google.com/uc?id=1eFvomqtNrGHz8eEEbF8NJo1qbLz7p5Pd" alt="drawing" width="800"/>




## [Linformer](https://arxiv.org/abs/2006.04768)
<img src="https://drive.google.com/uc?id=1BsOOaxA5JhVyx4K6bqqb66hKFDmmNX0g" alt="drawing" width="800"/>

<!-- ## [Performer](https://arxiv.org/abs/2009.14794)
### [Blog sobre Performer](https://chiaracampagnola.io/2020/10/29/from-transformers-to-performers/)

<img src="https://drive.google.com/uc?id=1xNmdUaMHWoCd4vXDdWm04tDexw_6U1O6" alt="drawing" width="200" height="228"/>
<img src="https://drive.google.com/uc?id=1e7oiJsyZmxnazCUyGE8Et4M6_buKuQyC" alt="drawing" width="600"/> -->


## [Reformer](https://arxiv.org/abs/2001.04451)
<img src="https://drive.google.com/uc?id=1LTXOUkGVAUAdOMApu7MVNp3GFwb3l8_J" alt="drawing" width="800"/>

## [BigBird](https://arxiv.org/abs/2007.14062)
### [Blog sobre BigBird](https://huggingface.co/blog/big-bird)

<!-- <img src="https://drive.google.com/uc?id=1jVnFPJek2TRhriy_yg3EkxCRI4uyjGpm" alt="drawing" width="400" height="250"/>

<img src="https://drive.google.com/uc?id=1S1T26KbwhQ-bmE84p7Ue9lyE1LURJFre" alt="drawing" width="400" height="250"/> -->

<img src="https://drive.google.com/uc?id=1RvzNUaXMPuvfgzd8cSOW-aU62dRt9jrP" alt="drawing" width="400" height="276.5"/>

<img src="https://drive.google.com/uc?id=12xz_X3ZLT81JRYbIuMDSU9h8tEe-kmaj" alt="drawing" width="400"/>

<img src="https://drive.google.com/uc?id=1kQUgdRx22Nr7NJGA25tls2nR5A07RTms" alt="drawing" width="802"/>

## Resultados: [Long Range Arena: A Benchmark for efficient transformers](https://arxiv.org/pdf/2011.04006.pdf)

<img src="https://drive.google.com/uc?id=1ruBhlaUSqIrFrD4k-K3d2rqZedSHMgEO" alt="drawing" width="802"/>

### [long-range-arena benchmark](https://github.com/google-research/long-range-arena)






## Em geral, as abordagens para o problema focam na redução de FLOPs e tendem a ignorar acessos à memória (**IO**).

<img src="https://drive.google.com/uc?id=1W_bwioQZ5ZO5Z5ex1wW0h_t6xhoTMon9" alt="drawing" width="800"/>



# ✨ 2a parte: Otimização Deep-Learning.


- ## Perspectiva (**inocente**): *Se eu pagar por **312** teraflops de computação, idealmente eu vou ter **312** teraflops.*
---

### Fonte: [Making Deep Learning Go Brrrr From First Principles](https://horace.io/brrr_intro.html)

> #### Se a computação é como uma fábrica... Enviamos instruções para nossa fábrica (**overhead**), enviamos materiais (**memory-bandwidth**), tudo para manter nossa fábrica funcionando com eficiência (**compute**).
<img src="https://drive.google.com/uc?id=1eQio68eJf7OJjrL5tvJBn-b2PxYYW9Mq" alt="drawing" width="600" height="300"/>

> #### Se nossa fábrica aumenta a eficiência mais rapidamente do que a taxa na qual podemos fornecer materiais, torna-se mais difícil para nossa fábrica atingir sua eficiência máxima.
<img src="https://drive.google.com/uc?id=1fsgkuvQcb_Yi99sig16daj66U1B8AKbT" alt="drawing" width="600" height="300"/>
>
> #### Mesmo que o tamanho de nossa fábrica (FLOPS) tenha dobrado - se nossa largura de banda (**bandwidth**) não puder acompanhar, nosso desempenho também não dobrará.

> #### GPUs têm hardware especializado para multiplicação de matrizes, como os "Tensor Cores" da Nvidia.
<img src="https://drive.google.com/uc?id=104GWauf8HugTLTuVCHv_Nw817wL9XFIj" alt="drawing" width="600" height="300"/>




> ### Artigo: [Data Movement Is All You Need: A Case Study on Optimizing Transformers](https://arxiv.org/abs/2007.00072)
> <img src="https://drive.google.com/uc?id=1lZbEEAVWcXA0nV8kJ3ZiGtVR1NfIq7ky" alt="drawing" width="800"/>
>
> #### Stat. normalization e Element-wise atingem **250x** menos FLOPS e **700x** menos FLOPS do que tensor contraction (matmuls), respectivamente.



# 🤔 Por que as operações "não matmul" levam muito mais tempo do que deveriam❓
- ### Voltando à nossa analogia, o culpado geralmente é o tempo que leva para transportar os materiais da e para a fábrica. Em outras palavras, a largura de banda da memória.


> <img src="https://drive.google.com/uc?id=1IPyU22at2Tb_uqRiVxq2SRYJpMYWdu-d" alt="drawing" width="600"/>

- ### Os custos de badwidhth são custos de mover dados da CPU para a GPU, de um nó para outro ou até mesmo da memória global CUDA para a memória compartilhada CUDA (***memory bandwidth cost***).
- ### A fábrica não deve ser usada como unidade de armazenamento, ela é eficiente para trabalho rápido e pequeno (**SRAM**).
- ### Tipicamente, o resultado é armazenado em algum armazem (**warehouse**): um local "mais barato" e com muito espaço (**DRAM** `nvidia-smi`). Dessa forma, podemos enviar e receber dados  da fábrica (**memory bandwidth**)
- ### Toda vez que executamos uma operação no kernel da GPU: precisamos mover o dado da  (**DRAM**).
- ### Por exemplo, a operação unária `torch.cos`: precisa enviar os dados para a warehouse, executar o cálculo para cada parte dos dados e, em seguida, enviar esse dado de volta. Como resultado, quase todo o nosso tempo aqui é gasto no transporte de dados, e não na computação.
- ### Como estamos gastando todo o nosso tempo na largura de banda da memória, essa operação é chamada de operação **memory-bound** e significa que não estamos gastando muito tempo em computação.



# Ok..., como perder menos tempo no transporte de dados da **DRAM** para a **SRAM**?

> <img src="https://drive.google.com/uc?id=1krlAoSySz-KN1aCR-5FIcdcrwZEvrD-o" alt="drawing" width="600"/>

### Operator fusion
> <img src="https://drive.google.com/uc?id=1TYoynirPaCv32wcEQ5q0jGflpEQ7anRi" alt="drawing" width="600"/>


# Overhead
> ### Overhead é quando seu código está gastando tempo fazendo qualquer coisa que não seja transferir tensores ou computar coisas.


### Python é lento...
> #### *Uma GPU A100 pode executar 312 trilhões de operações de ponto flutuante por segundo (312 TeraFLOPS). Em comparação,  pode realizar 32 milhões de adições em um segundo. Isso significa que, no tempo em que o Python pode executar um **único FLOP**, um A100 poderia ter executado **9,75 milhões de FLOPS**.*

# Operator Fusion
> ### lib para escrever CUDA kernels: [Introducing Triton: Open-source GPU programming for neural networks](https://openai.com/research/triton)


In [ ]:
import torch
x = torch.randn(1,256)
x.to("cuda")

# 4 global reads / writes
x1 = x.cos() # lê x da memória global, escreve em x1
x2 = x1.cos() # lê x1 da memória global, escreve em x2

# 2 global reads / writes
x2 = x.cos().cos() # lê x da memória global, escreve em x2

# ✨ 3a parte: Flash Attention $\;ELI\,5$

> ### [Artigo](https://arxiv.org/pdf/2205.14135.pdf)
> #### Blogpost [Aleksa Gordić](https://gordicaleksa.medium.com/eli5-flash-attention-5c44017022ad) 💪
> #### Blogpost [Memory Wall](https://www.semianalysis.com/p/nvidiaopenaitritonpytorch#%C2%A7the-memory-wall)
>
> <img src="https://drive.google.com/uc?id=139OP2I95uLUDxBsNkMimLooJYbpKb0sk" alt="drawing" width="1200"/>
---
## $Fast$:
> <img src="https://drive.google.com/uc?id=1oc5aS_gMsrLchYVtzd1cAW5Z3dIvkxxt" alt="drawing" width="1200"/>


## $Memory-Efficient$:
> <img src="https://drive.google.com/uc?id=1Wnrv3_OTH1-M_iyoVVDqa5-grZe0AhFx" alt="drawing" width="1200"/>

## $Exact$:
> <img src="https://drive.google.com/uc?id=1oJnXiC51fbSyhgAUJzDsogb5-X_zik5v" alt="drawing" width="1200"/>

## $IO-Awareness$:
> <img src="https://drive.google.com/uc?id=1or-Jen81qLiLdDuReoX2Jw68w7atWLdN" alt="drawing" width="1200"/>



## Voltando ao FLOPs...

> <img src="https://drive.google.com/uc?id=1U23BXqEi74xHIGWB5XiQjBFe4B92E7wg" alt="drawing" width="800" height="400"/>
---
## 🥴 Em Transformers, somente computação em velocidade de **exaFLOPs** não resolve! 🥴

## *IO-aware* matters!
- ## compute-bound
 - ### `matmul`
- ## memory-bound
 - ### `elementwise ops`: activation, dropout, masking
 - ### `reduction ops`: softmax, layer norm, sum, etc.
---
> <img src="https://drive.google.com/uc?id=1SmzCoBF3RO3v7eq5lUP2X9StGFAvAN_f" alt="drawing" width="800" height="600"/>
>

> ### A memória não é um artefato monolítico, é hierárquica em sua natureza e a regra geral é: *quanto mais rápida a memória, mais cara ela é e menor sua capacidade❗*
> <img src="https://drive.google.com/uc?id=17vUgAZ_rKdscBENONmDbFnvTz2kr6l_X" alt="drawing" width="800"/>





# Self Attention:
<img src="https://drive.google.com/uc?id=1OplMKDXvH_TtGLwbuO6O1t6PdUuX9PQV" alt="drawing" width="1200"/>

## Notation:
- Q — queries
- K — keys
- V — values
- S — scores
- P — probabilities
- O — outputs



In [75]:
import math
import torch
import torch.nn as nn

B = 1 # batch size
T = block_size = 4 # sequence length
C = n_embed = 6 # embedding dimensionality (n_embed)
n_head = 2 # num. heads

x = torch.randn(B, T, C)
c_attn = nn.Linear(C, 3*C)

q, k ,v  = c_attn(x).split(C, dim=2)
print(q.shape, k.shape, v.shape)

torch.Size([1, 4, 6]) torch.Size([1, 4, 6]) torch.Size([1, 4, 6])


In [76]:
Q = q.view(B, T, n_head, C//n_head).transpose(1, 2) # (B, n_head, T, head_size)
K = k.view(B, T, n_head, C//n_head).transpose(1, 2) # (B, n_head, T, head_size)
V = v.view(B, T, n_head, C//n_head).transpose(1, 2) # (B, n_head, T, head_size)

print(Q.shape, K.shape, V.shape)

torch.Size([1, 2, 4, 3]) torch.Size([1, 2, 4, 3]) torch.Size([1, 2, 4, 3])


In [77]:
# S=QK^T
S = torch.einsum("...th, ...lh -> ...tl", Q, K)  # (B, n_head, T, T)
S.shape

torch.Size([1, 2, 4, 4])

In [78]:
# É causal? Decoder?
Q_LEN = Q.shape[2]
K_LEN = K.shape[2]
causal_mask = torch.triu(torch.ones((Q_LEN, K_LEN)), K_LEN - Q_LEN + 1) # principal diag. will not be consider with +1
S = torch.where(causal_mask > 0, 1e-9, S)
S[0,0]

tensor([[-1.6420e-02,  1.0000e-09,  1.0000e-09,  1.0000e-09],
        [ 1.5213e+00,  1.1006e+00,  1.0000e-09,  1.0000e-09],
        [ 1.1597e+00, -2.5399e-01,  1.2816e+00,  1.0000e-09],
        [ 9.3451e-01,  1.0825e+00,  9.9049e-01,  1.1979e+00]],
       grad_fn=<SelectBackward0>)

In [79]:
# Probabilities P
P = nn.functional.softmax(S, dim=-1)
P[0,0]

tensor([[0.2469, 0.2510, 0.2510, 0.2510],
        [0.4777, 0.3136, 0.1043, 0.1043],
        [0.3722, 0.0905, 0.4205, 0.1167],
        [0.2213, 0.2566, 0.2341, 0.2880]], grad_fn=<SelectBackward0>)

In [80]:
# Outputs
O = P@V # B,n_head,T,T X B,n_head,T,head_size ---> B,n_head,T,head_size
O = O.transpose(1, 2).contiguous().view(B, T, C) # B,T,(n_head, head_size) ---> B,T,C
O.shape

torch.Size([1, 4, 6])

<img src="https://drive.google.com/uc?id=1OPiV2tLaIATK5rxudn-xepAym573C0Nb" alt="drawing" width="1200"/>


In [81]:
import numpy as np

BLOCK_SIZE = T
NEG_INF = -1e9 # -infinity
EPSILON = 1e-10

def flash_attention_causal_forward(Q, K, V):
    O = torch.zeros_like(Q, requires_grad=True)
    l = torch.zeros(Q.shape[:-1])[...,None]
    m = torch.ones(Q.shape[:-1])[...,None] * NEG_INF

    O = O.to(device='cuda')
    l = l.to(device='cuda')
    m = m.to(device='cuda')

    Q_BLOCK_SIZE = min(BLOCK_SIZE, Q.shape[-1])
    KV_BLOCK_SIZE = BLOCK_SIZE

    Q_BLOCKS = torch.split(Q, Q_BLOCK_SIZE, dim=2)
    K_BLOCKS = torch.split(K, KV_BLOCK_SIZE, dim=2)
    V_BLOCKS = torch.split(V, KV_BLOCK_SIZE, dim=2)

    Tr = len(Q_BLOCKS)
    Tc = len(K_BLOCKS)

    O_BLOCKS = list(torch.split(O, Q_BLOCK_SIZE, dim=2))
    l_BLOCKS = list(torch.split(l, Q_BLOCK_SIZE, dim=2))
    m_BLOCKS = list(torch.split(m, Q_BLOCK_SIZE, dim=2))

    Q_LEN = Q.shape[2]
    K_LEN = K.shape[2]

    Q_RANGE = torch.arange(Q_LEN)[:,None] + (K_LEN - Q_LEN)
    K_RANGE = torch.arange(K_LEN)[None,:]

    Q_RANGE = Q_RANGE.to(device='cuda')
    K_RANGE = K_RANGE.to(device='cuda')

    Q_RANGE_BLOCKS = torch.split(Q_RANGE, Q_BLOCK_SIZE, dim=0)
    K_RANGE_BLOCKS = torch.split(K_RANGE, KV_BLOCK_SIZE, dim=1)

    for j in range(Tc):
        Kj = K_BLOCKS[j]
        Vj = V_BLOCKS[j]
        K_RANGE_BLOCKSj = K_RANGE_BLOCKS[j]

        for i in range(Tr):
            Qi = Q_BLOCKS[i]
            Oi = O_BLOCKS[i]
            li = l_BLOCKS[i]
            mi = m_BLOCKS[i]
            Q_RANGE_BLOCKSi = Q_RANGE_BLOCKS[i]

            scale = 1 / np.sqrt(Q.shape[-1])
            Qi_scaled  = Qi * scale

            S_ij = torch.einsum('...id, ...jd -> ...ij', Qi_scaled, Kj)

            # Masking
            causal_mask = Q_RANGE_BLOCKSi >= K_RANGE_BLOCKSj
            S_ij = torch.where(causal_mask > 0, S_ij, NEG_INF)

            m_block_ij, _ = torch.max(S_ij, dim=-1, keepdims=True)
            P_ij = torch.exp(S_ij - m_block_ij)
            # Masking
            P_ij = torch.where(causal_mask > 0, P_ij, 0)

            l_block_ij = torch.sum(P_ij, dim=-1, keepdims=True) + EPSILON

            P_ij_Vj = torch.einsum('... i j, ... j d -> ... i d', P_ij, Vj)

            mi_new = torch.maximum(m_block_ij, mi)
            li_new = torch.exp(mi - mi_new) * li + torch.exp(m_block_ij - mi_new) * l_block_ij

            O_BLOCKS[i] = (li/li_new) * torch.exp(mi - mi_new) * Oi + (torch.exp(m_block_ij - mi_new) / li_new) * P_ij_Vj
            l_BLOCKS[i] = li_new
            m_BLOCKS[i] = mi_new

    O = torch.cat(O_BLOCKS, dim=2)
    l = torch.cat(l_BLOCKS, dim=2)
    m = torch.cat(m_BLOCKS, dim=2)
    return O, l, m

def flash_attention_causal_backward(Q, K, V, O, l, m, dO):
    Q_BLOCK_SIZE = min(BLOCK_SIZE, Q.shape[-1])
    KV_BLOCK_SIZE = BLOCK_SIZE

    Q_BLOCKS = torch.split(Q, Q_BLOCK_SIZE, dim=2)
    K_BLOCKS = torch.split(K, KV_BLOCK_SIZE, dim=2)
    V_BLOCKS = torch.split(V, KV_BLOCK_SIZE, dim=2)

    Tr = len(Q_BLOCKS)
    Tc = len(K_BLOCKS)

    O_BLOCKS = list(torch.split(O, Q_BLOCK_SIZE, dim=2))
    dO_BLOCKS = list(torch.split(dO, Q_BLOCK_SIZE, dim=2))
    l_BLOCKS = list(torch.split(l, Q_BLOCK_SIZE, dim=2))
    m_BLOCKS = list(torch.split(m, Q_BLOCK_SIZE, dim=2))

    dQ = torch.zeros_like(Q, requires_grad=True).to(device='cuda')
    dK = torch.zeros_like(K, requires_grad=True).to(device='cuda')
    dV = torch.zeros_like(V, requires_grad=True).to(device='cuda')

    dQ_BLOCKS = list(torch.split(dQ, Q_BLOCK_SIZE, dim=2))
    dK_BLOCKS = list(torch.split(dK, KV_BLOCK_SIZE, dim=2))
    dV_BLOCKS = list(torch.split(dV, KV_BLOCK_SIZE, dim=2))

    Q_LEN = Q.shape[2]
    K_LEN = K.shape[2]

    Q_RANGE = torch.arange(Q_LEN)[:,None] + (K_LEN - Q_LEN)
    K_RANGE = torch.arange(K_LEN)[None,:]

    Q_RANGE = Q_RANGE.to(device='cuda')
    K_RANGE = K_RANGE.to(device='cuda')

    Q_RANGE_BLOCKS = torch.split(Q_RANGE, Q_BLOCK_SIZE, dim=0)
    K_RANGE_BLOCKS = torch.split(K_RANGE, KV_BLOCK_SIZE, dim=1)

    for j in range(Tc):
        Kj = K_BLOCKS[j]
        Vj = V_BLOCKS[j]
        K_RANGE_BLOCKSj = K_RANGE_BLOCKS[j]

        dKj_block = torch.zeros_like(dK_BLOCKS[j], requires_grad=True).to(device='cuda')
        dVj_block = torch.zeros_like(dV_BLOCKS[j], requires_grad=True).to(device='cuda')

        for i in range(Tr):
            Qi = Q_BLOCKS[i]
            Oi = O_BLOCKS[i]
            dOi = dO_BLOCKS[i]
            li = l_BLOCKS[i]
            mi = m_BLOCKS[i]
            Q_RANGE_BLOCKSi = Q_RANGE_BLOCKS[i]

            scale = 1 / np.sqrt(Q.shape[-1])
            Qi_scaled  = Qi * scale

            S_ij = torch.einsum('... i d, ... j d -> ... i j', Qi_scaled, Kj)

            # Masking
            causal_mask = Q_RANGE_BLOCKSi >= K_RANGE_BLOCKSj
            S_ij = torch.where(causal_mask > 0, S_ij, NEG_INF)

            P_ij = (1/li) * torch.exp(S_ij - mi)
            # Masking
            P_ij = torch.where(causal_mask > 0, P_ij, 0)

            dVj_block = dVj_block + torch.einsum('... r c, ... r d -> ... c d', P_ij, dOi)
            dP_ij = torch.einsum('... r d, ... c d -> ... r c', dOi, Vj)

            Di = torch.sum(dOi * Oi, dim=-1, keepdims=True)
            dS_ij = P_ij * (dP_ij - Di)

            dQ_BLOCKS[i] = dQ_BLOCKS[i] + scale * torch.einsum('... r c, ... c d -> ... r d', dS_ij, Kj)

            dKj_block = dKj_block + scale * torch.einsum('... r c, ... r d -> ... c d', dS_ij, Qi)

        dK_BLOCKS[j] = dKj_block
        dV_BLOCKS[j] = dVj_block

    dQ = torch.cat(dQ_BLOCKS, dim=2)
    dK = torch.cat(dK_BLOCKS, dim=2)
    dV = torch.cat(dV_BLOCKS, dim=2)
    return dQ, dK, dV

def flash_attention_causal(Q, K, V):
    out = flash_attention_causal_forward(Q, K, V)
    return out[0]

def normal_attention_causal(Q, K, V):
    scale = 1 / np.sqrt(Q.shape[-1])
    Q = Q * scale
    QKt = torch.einsum('... i d, ... j d -> ... i j', Q, K)

    Q_LEN = Q.shape[2]
    K_LEN = K.shape[2]

    causal_mask = torch.triu(torch.ones((Q_LEN, K_LEN)), K_LEN - Q_LEN + 1)
    causal_mask = causal_mask.to(device='cuda')
    QKt = torch.where(causal_mask > 0, NEG_INF, QKt)

    attn = nn.functional.softmax(QKt, dim=-1)
    return attn @ V

In [86]:
for i in range(10):
    flash_attn = flash_attention_causal(Q.to('cuda'), K.to('cuda'), V.to('cuda'))

    normal_attn = normal_attention_causal(Q.to('cuda'), K.to('cuda'), V.to('cuda'))

    print(torch.allclose(flash_attn, normal_attn), end=' ---> ')

True ---> True ---> True ---> True ---> True ---> True ---> True ---> True ---> True ---> True ---> 

In [ ]:
#Causal mask FlashAttention

466.40176ms
STAGE:2023-06-25 14:50:39 787:787 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-06-25 14:50:40 787:787 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-06-25 14:50:40 787:787 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  Total MFLOPs  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------ 

In [ ]:
#Normal attention

991.550615ms
STAGE:2023-06-25 14:51:03 866:866 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-06-25 14:51:04 866:866 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-06-25 14:51:04 866:866 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  Total MFLOPs  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------